### Type Hinting
Please use type hints wherever possible. This will help improve the readability and maintainability of your code.

Incorrect or incomplete type hinting **will** result in penalties if trivial type hints are used incorrectly (for example, `list` is used instead of `float`, where the variable is obviously a number).




## Problem 1 [25 points]

Create an abstract base class `Expression` that represents a mathematical expression. This class should require methods for evaluating the expression (given a dictionary of variable values), for computing its symbolic derivative with respect to a variable. Also, it should require pretty printing and formatting for expressions. Then, implement concrete classes for constants, variables and binary operations (addition, subtraction, multiplication and division).

In [14]:
from abc import ABC, abstractmethod


class Expression(ABC):

    @abstractmethod
    def evaluate(self, values_dict: dict):
        pass

    @abstractmethod
    def derivative(self, with_respect_to: str):
        pass


class Constant(Expression):
    def __init__(self, value):
        self.value = value

    def evaluate(self, values_dict={}):
        return self.value

    def __repr__(self):
        return f"Constant(value={self.value})"

    def __str__(self):
        return str(self.value)
    
    def derivative(self, respect_to: str):
        return Constant(0)


class Variable(Expression):
    def __init__(self, name):
        self.name = name

    def evaluate(self, x_str):
        return x_str[self.name]

    def derivative(self, respect_to: str):
        if respect_to == self.name:
            return Constant(1)
        else:
            return Constant(0)

    def __repr__(self):
        return f"Variable(name={self.name})"

    def __str__(self):
        return str(self.name)


class Add(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) + self.expr_2.evaluate(values_dict)
    
    def derivative(self, respect_to: str):
        return Add(self.expr_1.derivative(respect_to), self.expr_2.derivative(respect_to))

    def __repr__(self):
        return f"Add(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"

    def __str__(self):
        return f"({self.expr_1.__str__()} + {self.expr_2.__str__()})"


class Subtract(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) - self.expr_2.evaluate(values_dict)


    def derivative(self, respect_to: str):
        return Subtract(self.expr_1.derivative(respect_to), self.expr_2.derivative(respect_to))

    def __repr__(self):
        return (
            f"Subtract(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"
        )

    def __str__(self):
        return f"({self.expr_1.__str__()} - {self.expr_2.__str__()})"


class Multiply(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) * self.expr_2.evaluate(values_dict)

    def derivative(self, respect_to: str):
        first_part = Multiply(self.expr_1.derivative(respect_to), self.expr_2)
        second_part = Multiply(self.expr_1, self.expr_2.derivative(respect_to))

        return Add(first_part, second_part)

    def __repr__(self):
        return (
            f"Multiply(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"
        )

    def __str__(self):
        if isinstance(self.expr_1, Constant | Variable) or isinstance(self.expr_2, Constant | Variable):
            return f"{self.expr_1.__str__()} * {self.expr_2.__str__()}"
        else:
            return f"({self.expr_1.__str__()} * {self.expr_2.__str__()})"

class Divide(Expression):
    def __init__(self, expr_1: Expression, expr_2: Expression):
        self.expr_1 = expr_1
        self.expr_2 = expr_2

    def evaluate(self, values_dict: dict):
        return self.expr_1.evaluate(values_dict) / self.expr_2.evaluate(values_dict)
    
    def derivative(self, respect_to: str):
        first_part = Multiply(self.expr_1.derivative(respect_to), self.expr_2)
        second_part = Multiply(self.expr_1, self.expr_2.derivative(respect_to))


        return Divide(Subtract(first_part, second_part), Multiply(self.expr_2, self.expr_2))

    def __repr__(self):
        return (
            f"Divide(expr_1={self.expr_1.__str__()}, expr_2={self.expr_2.__str__()})"
        )

    def __str__(self):
        return f"{self.expr_1.__str__()} / {self.expr_2.__str__()}"


expr = Divide(
    Multiply(
        Add(Multiply(Constant(2), Variable("x")), Constant(3)),
        Subtract(Variable("x"), Constant(5))
    ),
    Subtract(Constant(1), Variable("x"))
)

print(expr) # (2 * x + 3) * (x - 5) / (1 - x)

assert expr.evaluate({"x": 3}) == 9.0

derivative_expr = expr.derivative("x")

assert derivative_expr.evaluate({"x": 3}) == -7.0

((2 * x + 3) * (x - 5)) / (1 - x)


## Problem 2 [25 points]


A Binary Search Tree (BST) is a data structure that consists of nodes, where each node contains a value and references to two child nodes: left and right. The key property of a BST is that for every node:

- The value of the left child node is less than the value of the parent node.
- The value of the right child node is greater than the value of the parent node.

Implement a generic BST that supports any comparable type (integers, floats, strings, custom objects). It should provide standard `insert` and `search` operations as defined below:

- **Insertion:** Insert a new node while maintaining the BST property.
- **Search:** Find a node in the tree based on its value.


In [41]:
from typing import TypeVar, Generic

T = TypeVar("T")


class BSTNode(Generic[T]):
    def __init__(self, value: T, left_ref=None, right_ref=None):
        self.value = value
        self.left_ref = left_ref
        self.right_ref = right_ref


class BST(Generic[T]):
    def __init__(self):
        self.root_node = None

    def insert(self, value):
        new_node = BSTNode(value=value)
        curr_node = self.root_node
        if self.root_node:
            while True:
                if value > curr_node.value:
                    if curr_node.right_ref:
                        curr_node = curr_node.right_ref
                    else:
                        curr_node.right_ref = new_node
                        break

                else:
                    if curr_node.left_ref:
                        curr_node = curr_node.left_ref
                    else:
                        curr_node.left_ref = new_node
                        break
        else:
            self.root_node = new_node

    def search(self, value):
        curr_node = self.root_node
        if self.root_node:
            while True:
                if value > curr_node.value:
                    if curr_node.right_ref:
                        curr_node = curr_node.right_ref
                    else:
                        return False

                elif value < curr_node.value:
                    if curr_node.left_ref:
                        curr_node = curr_node.left_ref
                    else:
                        return False
                else:
                    return True


"""
        10
       /  \
     5     15
    / \    /  \
   3   7  12   20
"""

tree = BST[int]()
tree.insert(10)
tree.insert(5)
tree.insert(15)
tree.insert(3)
tree.insert(7)
tree.insert(12)
tree.insert(20)

assert tree.search(7) == True
assert tree.search(15) == True
assert tree.search(42) == False

<>:57: SyntaxWarning: invalid escape sequence '\ '
<>:57: SyntaxWarning: invalid escape sequence '\ '
/var/folders/px/bf2tz3gn0sjd3g4js54j6gd80000gn/T/ipykernel_30191/1738458969.py:57: SyntaxWarning: invalid escape sequence '\ '
  """


## Problem 3 [25 points]

### a. Implement a descriptor that tracks the history of all values assigned to an attribute.

In [128]:
from typing import Any


class HistoryTracking:
    def __set_name__(self, owner: type, name: str) -> None:
        self.name = name
        hist_attr = f"_{name}_history"

        def get_history(self) -> list[Any]:
            return list(getattr(self, hist_attr, []))

        setattr(owner, f"get_{name}_history", get_history)

    def __set__(self, instance, value):
        hist_attr = f"_{self.name}_history"
        if self.name in instance.__dict__:
            instance.__dict__.setdefault(hist_attr, []).append(
                instance.__dict__[self.name]
            )

        instance.__dict__[self.name] = value

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return instance.__dict__.get(self.name, None)

    def __delete__(self, instance):
        del instance.__dict__[self.name]


class StockPrice:
    price = HistoryTracking()

    def __init__(self, stock_name, price_value):
        self.stock_name = stock_name
        self.price = price_value


stock = StockPrice("AAPL", 200)

assert stock.price == 200

stock.price = 210

assert stock.price == 210

stock.price = 225

assert stock.price == 225
assert stock.get_price_history() == [200, 210]

stock.price = 215

assert stock.price == 215
assert stock.get_price_history() == [200, 210, 225]

### b. Implement a descriptor that logs the access to an attribute, along with the time and value, into a log file.

In [ ]:
import time


class FileLoggingDescriptor:
    def __init__(self, log_file: str):
        self.log_file = log_file

    def __set_name__(self, owner: type, name: str) -> None:
        self.name = name

    def __get__(self, instance, owner):
        with open(self.log_file, "a") as f:
            f.write(
                f"{time.strftime("%a %b %d %H:%M:%S %Y:")} Accessed value {instance.__dict__.get(self.name, None)}\n"
            )
        return instance.__dict__.get(self.name, None)

    def __set__(self, instance, value):
        with open(self.log_file, "a") as f:
            f.write(
                f"{time.strftime('%a %b %d %H:%M:%S %Y:')} Modified value from {instance.__dict__.get(self.name, 'no value')} to {value}\n"
            )

        instance.__dict__[self.name] = value


class Person:
    name = FileLoggingDescriptor(log_file="access_log.txt")

    def __init__(self, name):
        self.name = name


obj = Person("John Doe")

print(obj.name)  # "John Doe"

obj.name = "Jane Dane"

print(obj.name)  # "Jane Dane"

with open("access_log.txt", "r") as f:
    print(f.read())

"""
Sat Mar 29 20:23:56 2025: Modified value from 'no value' to 'John Doe'
Sat Mar 29 20:23:56 2025: Accessed value 'John Doe'
Sat Mar 29 20:23:56 2025: Modified value from 'John Doe' to 'Jane Dane'
Sat Mar 29 20:23:56 2025: Accessed value 'Jane Dane'
"""

John Doe
Jane Dane
Sun Apr 20 22:01:25 2025: Modified value from no value to John Doe
Sun Apr 20 22:01:25 2025: Accessed value John Doe
Sun Apr 20 22:01:25 2025: Modified value from John Doe to Jane Dane
Sun Apr 20 22:01:25 2025: Accessed value Jane Dane



"\nSat Mar 29 20:23:56 2025: Modified value from 'no value' to 'John Doe'\nSat Mar 29 20:23:56 2025: Accessed value 'John Doe'\nSat Mar 29 20:23:56 2025: Modified value from 'John Doe' to 'Jane Dane'\nSat Mar 29 20:23:56 2025: Accessed value 'Jane Dane'\n"

## Problem 4 [25 points]


Write a function `build_logging_class` that dynamically generates a new class which wraps all public methods of a given base class with logging behavior.

The logging should:

- Print a log message before the method is called, showing its name and arguments.
- Print a log message after the method is called, showing the result.
- Exclude dunder methods (`__init__`, `__str__`, etc.).
- Allow injection of a custom logging function (defaults to print).

#### (Optional) Bonus requirements

- Support a logging level prefix like [ERROR], [INFO], [DEBUG].
- Allow logging to be written to a file.
- Support toggling logging on/off.


In [ ]:
from typing import Callable
import functools
import logging
import inspect

def get_public_methods(cls):
    all_funcs = inspect.getmembers(cls, predicate=inspect.isfunction)
    public = {
        name: func for name, func in all_funcs
        if not (name.startswith("__") and name.endswith("__"))
    }
    return public

def make_wrapper(fn, log_fn):
    @functools.wraps(fn)
    def wrapped(*args, **kwargs):
        log_fn(f"[LOG] Calling: `{fn.__name__}` with args={args[1:]} kwargs={kwargs}")
        out = fn(*args, **kwargs)
        log_fn(f"[LOG] Result of `{fn.__name__}` {out}")
        return out
    return wrapped


def build_logging_class(name: str, base_class: type, log_func: Callable = print) -> type:
    new_func_list = []
    public_fns = get_public_methods(base_class)
    wrapped_funcs = {name: make_wrapper(fn, log_func) for name, fn in public_fns.items()}
    return type(name, (base_class, ), wrapped_funcs)

        


class Calculator:
    def add(self, x, y): return x + y
    def mul(self, x, y): return x * y

LoggingCalculator = build_logging_class("LoggingCalculator", Calculator)

calc = LoggingCalculator()
calc.add(2, 3)
calc.mul(4, 5)

"""
Expected output

[LOG] Calling: `add` with args=(2, 3) kwargs={}
[LOG] Result of `add`: 5
[LOG] Calling: `mul` with args=(4, 5) kwargs={}
[LOG] Result of `mul`: 20
"""


[LOG] Calling: `add` with args=(2, 3) kwargs={}
[LOG] Result of `add` 5
[LOG] Calling: `mul` with args=(4, 5) kwargs={}
[LOG] Result of `mul` 20


'\nExpected output\n\n[LOG] Calling: `add` with args=(2, 3) kwargs={}\n[LOG] Result of `add`: 5\n[LOG] Calling: `mul` with args=(4, 5) kwargs={}\n[LOG] Result of `mul`: 20\n'